In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icassp-person-in-bed-track-1/sample_submission.csv
/kaggle/input/icassp-person-in-bed-track-1/train.json
/kaggle/input/icassp-person-in-bed-track-1/License Agreement.pdf
/kaggle/input/icassp-person-in-bed-track-1/test.json


# Read csv

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

with open('/kaggle/input/icassp-person-in-bed-track-1/train.json', 'r') as f:
    data = json.load(f)


df = pd.DataFrame(data)

In [7]:
import json
import numpy as np
from scipy.signal import butter, filtfilt

# Define a function to create a low-pass Butterworth filter
def butter_lowpass(cutoff, fs, order=4):
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist  # Normalize cutoff frequency
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# Apply the low-pass filter to data
def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order)
    return filtfilt(b, a, data)

# Function to process a single subject
def process_subject(subject_data, cutoff_freq=45.0, sampling_rate=250):
    # Extract accelerometer data and timestamps
    accel_data = np.array(subject_data['accel'])
    ts_data = np.array(subject_data['ts'])
    
    # Apply low-pass filter to each axis (x, y, z)
    filtered_accel = np.array([butter_lowpass_filter(accel_data[:, i], cutoff_freq, sampling_rate) for i in range(3)]).T
    
    # Return processed subject data without downsampling
    return {
        'subject': subject_data['subject'],
        'accel': filtered_accel.tolist(),
        'ts': ts_data.tolist(),
        'labels': subject_data['label']  # Keep original labels
    }

# Load the JSON file
with open('/kaggle/input/icassp-person-in-bed-track-1/train.json', 'r') as f:
    data = json.load(f)

# Process each subject independently
processed_data = [process_subject(subject_data) for subject_data in data]

# Save the processed data back to JSON
with open('LowPass_train_45.json', 'w') as f:
    json.dump(processed_data, f)

print("Low-pass filtering (15 Hz) complete without downsampling.")

Low-pass filtering (15 Hz) complete without downsampling.


In [3]:
len(df),df.shape

(5911, (5911, 5))

In [12]:
# Split the DataFrame
df1 = df.iloc[:4728]  # First 4728 rows
df2 = df.iloc[4728:]  # Remaining rows

In [19]:
df1.shape,df2.shape

((4728, 5), (1183, 5))

4728

In [4]:
df.head()

,subject,chunk_id,ts,accel,label
0,80,0,"[1725411020.0856903, 1725411020.0897202, 17254...","[[-82.5083999633789, -56.03909683227539, 1000....",0
1,14,1,"[1725496482.262797, 1725496482.266831, 1725496...","[[-20.9507999420166, 65.57850646972656, 1027.9...",1
2,300,2,"[1726611768.779811, 1726611768.783809, 1726611...","[[108.17430114746094, 173.58509826660156, 968....",1
3,193,3,"[1725492595.229943, 1725492595.233981, 1725492...","[[15.732599258422852, 70.74210357666016, 1001....",1
4,9,4,"[1725486791.803703, 1725486791.807745, 1725486...","[[-29.815500259399414, 47.252403259277344, 100...",0


In [5]:
df['ts'].iloc[:1],len(df['ts'].iloc[:1])
cnt = 0
for i in df['ts'].iloc[:1]:
    for j in i:
        # print(j)
        cnt += 1
print("cnt = ",cnt)

# print(len(df['accel'].iloc[:1]))
# cnt = 0
# for i in df['accel'].iloc[:1]:
#     for j in i:
#         print(j)
#         cnt += 1
# print("cnt = ",cnt)

cnt =  2500


In [8]:
df['accel'].iloc[:1]

0    [[-82.5083999633789, -56.03909683227539, 1000....
Name: accel, dtype: object

## FFT

In [ ]:
def extract_periodic_features(accel_data):
    # Convert to numpy array for calculations
    accel_array = np.array(accel_data)
    x, y, z = accel_array[:, 0], accel_array[:, 1], accel_array[:, 2]
    
    # Compute FFT
    fft_x = np.fft.fft(x)
    fft_y = np.fft.fft(y)
    fft_z = np.fft.fft(z)
    
    # Compute the frequency bins
    n = len(x)
    freqs = np.fft.fftfreq(n)

    # Calculate the amplitude spectrum
    amplitude_x = np.abs(fft_x)
    amplitude_y = np.abs(fft_y)
    amplitude_z = np.abs(fft_z)

    # Extract features (e.g., dominant frequency)
    dominant_freq_x = freqs[np.argmax(amplitude_x[1:n//2])]  # Exclude the zero frequency
    dominant_freq_y = freqs[np.argmax(amplitude_y[1:n//2])]
    dominant_freq_z = freqs[np.argmax(amplitude_z[1:n//2])]
    
    return {
        'dominant_freq_x': dominant_freq_x,
        'dominant_freq_y': dominant_freq_y,
        'dominant_freq_z': dominant_freq_z,
        'mean_x': np.mean(x),
        'mean_y': np.mean(y),
        'mean_z': np.mean(z),
    }

In [ ]:
feature_list = df['accel'].apply(extract_periodic_features).tolist()
features_df = pd.DataFrame(feature_list)

# Combine features with labels
final_df = pd.concat([features_df, df['label']], axis=1)

print(final_df.head())

# Wavelet Transformation

## 4 features

In [29]:
import pywt
import numpy as np

def extract_wavelet_features_old(accel_data, wavelet='db4', level=4):
    """
    Extract features using wavelet transformation on accelerometer data.
    
    Parameters:
        accel_data: list or numpy array of accelerometer data (Nx3)
                   (where N is the number of samples, and 3 refers to x, y, z axes)
        wavelet: Wavelet type to use (default: 'db4')
        level: Decomposition level (default: 4)
    
    Returns:
        A dictionary containing wavelet features for x, y, z axes.
    """
    # Convert to numpy array for calculations
    accel_array = np.array(accel_data)
    x, y, z = accel_array[:, 0], accel_array[:, 1], accel_array[:, 2]

    # Perform discrete wavelet transform for each axis
    coeffs_x = pywt.wavedec(x, wavelet, level=level)
    coeffs_y = pywt.wavedec(y, wavelet, level=level)
    coeffs_z = pywt.wavedec(z, wavelet, level=level)

    # Helper function to summarize wavelet coefficients
    def summarize_coeffs(coeffs):
        return {
            'mean': np.mean(coeffs),
            'std': np.std(coeffs),
            'max': np.max(coeffs),
            'min': np.min(coeffs),
            'energy': np.sum(np.square(coeffs)),
            'skewness': np.mean((coeffs - np.mean(coeffs))**3) / (np.std(coeffs)**3 + 1e-6),
            'kurtosis': np.mean((coeffs - np.mean(coeffs))**4) / (np.std(coeffs)**4 + 1e-6),
        }

    # Extract features for approximation (cA) and detail (cD) coefficients
    features = {}
    for i, (cA_x, cA_y, cA_z) in enumerate(zip(coeffs_x, coeffs_y, coeffs_z)):
        features.update({
            f'cA_mean_x_level{i}': summarize_coeffs(cA_x)['mean'],
            f'cA_std_x_level{i}': summarize_coeffs(cA_x)['std'],
            f'cA_max_x_level{i}': summarize_coeffs(cA_x)['max'],
            f'cA_energy_x_level{i}': summarize_coeffs(cA_x)['energy'],
            f'cA_mean_y_level{i}': summarize_coeffs(cA_y)['mean'],
            f'cA_std_y_level{i}': summarize_coeffs(cA_y)['std'],
            f'cA_max_y_level{i}': summarize_coeffs(cA_y)['max'],
            f'cA_energy_y_level{i}': summarize_coeffs(cA_y)['energy'],
            f'cA_mean_z_level{i}': summarize_coeffs(cA_z)['mean'],
            f'cA_std_z_level{i}': summarize_coeffs(cA_z)['std'],
            f'cA_max_z_level{i}': summarize_coeffs(cA_z)['max'],
            f'cA_energy_z_level{i}': summarize_coeffs(cA_z)['energy'],
        })

    return features


## More features

In [8]:
import pywt
import numpy as np
from scipy.stats import skew, kurtosis, entropy

def extract_wavelet_features(accel_data, wavelet='db4', level=2):
    """
    Extract features using wavelet transformation on accelerometer data.
    
    Parameters:
        accel_data: list or numpy array of accelerometer data (Nx3)
                   (where N is the number of samples, and 3 refers to x, y, z axes)
        wavelet: Wavelet type to use (default: 'db4')
        level: Decomposition level (default: 6)
    
    Returns:
        A dictionary containing wavelet features for x, y, z axes.
    """
    # Convert to numpy array for calculations
    accel_array = np.array(accel_data)
    x, y, z = accel_array[:, 0], accel_array[:, 1], accel_array[:, 2]

    # Perform discrete wavelet transform for each axis
    coeffs_x = pywt.wavedec(x, wavelet, level=level)
    coeffs_y = pywt.wavedec(y, wavelet, level=level)
    coeffs_z = pywt.wavedec(z, wavelet, level=level)

    # Helper function to summarize wavelet coefficients
    def summarize_coeffs(coeffs):
        freqs = np.fft.fftfreq(len(coeffs))  # Frequency domain information
        fft_coeffs = np.fft.fft(coeffs)      # FFT of the coefficients
        amplitude_spectrum = np.abs(fft_coeffs)
        dominant_freq = freqs[np.argmax(amplitude_spectrum[1:])] if len(freqs) > 1 else 0
        
        return {
            'mean': np.mean(coeffs),
            'std': np.std(coeffs),
            'max': np.max(coeffs),
            'min': np.min(coeffs),
            'energy': np.sum(np.square(coeffs)),
            'skewness': skew(coeffs),
            'kurtosis': kurtosis(coeffs),
            'median': np.median(coeffs),
            'iqr': np.percentile(coeffs, 75) - np.percentile(coeffs, 25),
            'entropy': entropy(np.abs(coeffs) + 1e-6),  # Add small value to avoid log(0)
            'dominant_frequency': dominant_freq
        }

    # Extract features for approximation (cA) and detail (cD) coefficients
    features = {}
    for i, (cA_x, cA_y, cA_z) in enumerate(zip(coeffs_x, coeffs_y, coeffs_z)):
        features.update({
            f'cA_mean_x_level{i}': summarize_coeffs(cA_x)['mean'],
            f'cA_std_x_level{i}': summarize_coeffs(cA_x)['std'],
            f'cA_max_x_level{i}': summarize_coeffs(cA_x)['max'],
            f'cA_min_x_level{i}': summarize_coeffs(cA_x)['min'],
            f'cA_energy_x_level{i}': summarize_coeffs(cA_x)['energy'],
            f'cA_skewness_x_level{i}': summarize_coeffs(cA_x)['skewness'],
            f'cA_kurtosis_x_level{i}': summarize_coeffs(cA_x)['kurtosis'],
            f'cA_median_x_level{i}': summarize_coeffs(cA_x)['median'],
            f'cA_iqr_x_level{i}': summarize_coeffs(cA_x)['iqr'],
            f'cA_entropy_x_level{i}': summarize_coeffs(cA_x)['entropy'],
            f'cA_dominant_frequency_x_level{i}': summarize_coeffs(cA_x)['dominant_frequency'],

            f'cA_mean_y_level{i}': summarize_coeffs(cA_y)['mean'],
            f'cA_std_y_level{i}': summarize_coeffs(cA_y)['std'],
            f'cA_max_y_level{i}': summarize_coeffs(cA_y)['max'],
            f'cA_min_y_level{i}': summarize_coeffs(cA_y)['min'],
            f'cA_energy_y_level{i}': summarize_coeffs(cA_y)['energy'],
            f'cA_skewness_y_level{i}': summarize_coeffs(cA_y)['skewness'],
            f'cA_kurtosis_y_level{i}': summarize_coeffs(cA_y)['kurtosis'],
            f'cA_median_y_level{i}': summarize_coeffs(cA_y)['median'],
            f'cA_iqr_y_level{i}': summarize_coeffs(cA_y)['iqr'],
            f'cA_entropy_y_level{i}': summarize_coeffs(cA_y)['entropy'],
            f'cA_dominant_frequency_y_level{i}': summarize_coeffs(cA_y)['dominant_frequency'],

            f'cA_mean_z_level{i}': summarize_coeffs(cA_z)['mean'],
            f'cA_std_z_level{i}': summarize_coeffs(cA_z)['std'],
            f'cA_max_z_level{i}': summarize_coeffs(cA_z)['max'],
            f'cA_min_z_level{i}': summarize_coeffs(cA_z)['min'],
            f'cA_energy_z_level{i}': summarize_coeffs(cA_z)['energy'],
            f'cA_skewness_z_level{i}': summarize_coeffs(cA_z)['skewness'],
            f'cA_kurtosis_z_level{i}': summarize_coeffs(cA_z)['kurtosis'],
            f'cA_median_z_level{i}': summarize_coeffs(cA_z)['median'],
            f'cA_iqr_z_level{i}': summarize_coeffs(cA_z)['iqr'],
            f'cA_entropy_z_level{i}': summarize_coeffs(cA_z)['entropy'],
            f'cA_dominant_frequency_z_level{i}': summarize_coeffs(cA_z)['dominant_frequency'],
        })

    return features


## Both approx and detial components

In [5]:
import numpy as np
import pywt
from scipy.stats import skew, kurtosis
from scipy.stats import entropy

def extract_wavelet_features(accel_data, wavelet='db4', level=2):
    """
    Extract features using wavelet transformation on accelerometer data.
    
    Parameters:
        accel_data: list or numpy array of accelerometer data (Nx3)
                   (where N is the number of samples, and 3 refers to x, y, z axes)
        wavelet: Wavelet type to use (default: 'db4')
        level: Decomposition level (default: 6)
    
    Returns:
        A dictionary containing wavelet features for x, y, z axes.
    """
    # Convert to numpy array for calculations
    accel_array = np.array(accel_data)
    x, y, z = accel_array[:, 0], accel_array[:, 1], accel_array[:, 2]

    # Perform discrete wavelet transform for each axis
    coeffs_x = pywt.wavedec(x, wavelet, level=level)
    coeffs_y = pywt.wavedec(y, wavelet, level=level)
    coeffs_z = pywt.wavedec(z, wavelet, level=level)

    # Helper function to summarize wavelet coefficients
    def summarize_coeffs(coeffs):
        freqs = np.fft.fftfreq(len(coeffs))  # Frequency domain information
        fft_coeffs = np.fft.fft(coeffs)      # FFT of the coefficients
        amplitude_spectrum = np.abs(fft_coeffs)
        dominant_freq = freqs[np.argmax(amplitude_spectrum[1:])] if len(freqs) > 1 else 0
        
        return {
            'mean': np.mean(coeffs),
            'std': np.std(coeffs),
            'max': np.max(coeffs),
            'min': np.min(coeffs),
            'energy': np.sum(np.square(coeffs)),
            'skewness': skew(coeffs),
            'kurtosis': kurtosis(coeffs),
            'median': np.median(coeffs),
            'iqr': np.percentile(coeffs, 75) - np.percentile(coeffs, 25),
            'entropy': entropy(np.abs(coeffs) + 1e-6),  # Add small value to avoid log(0)
            'dominant_frequency': dominant_freq
        }

    # Extract features for both approximation (cA) and detail (cD) coefficients
    features = {}
    for i, (coeff_x, coeff_y, coeff_z) in enumerate(zip(coeffs_x, coeffs_y, coeffs_z)):
        # Determine component type: cA (approximation) or cD (detail)
        component = 'cA' if i == 0 else f'cD{i}'

        # Extract features for each axis and component
        for axis, coeffs in zip(['x', 'y', 'z'], [coeff_x, coeff_y, coeff_z]):
            summary = summarize_coeffs(coeffs)
            features.update({
                f'{component}_mean_{axis}_level{i}': summary['mean'],
                f'{component}_std_{axis}_level{i}': summary['std'],
                f'{component}_max_{axis}_level{i}': summary['max'],
                f'{component}_min_{axis}_level{i}': summary['min'],
                f'{component}_energy_{axis}_level{i}': summary['energy'],
                f'{component}_skewness_{axis}_level{i}': summary['skewness'],
                f'{component}_kurtosis_{axis}_level{i}': summary['kurtosis'],
                f'{component}_median_{axis}_level{i}': summary['median'],
                f'{component}_iqr_{axis}_level{i}': summary['iqr'],
                f'{component}_entropy_{axis}_level{i}': summary['entropy'],
                f'{component}_dominant_frequency_{axis}_level{i}': summary['dominant_frequency'],
            })

    return features


# Wavelet Packet Transformation

In [44]:
import numpy as np
import pandas as pd
import pywt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Example accelerometer data (Nx3 for x, y, z axes)
def generate_dummy_data(n_samples, n_features=3):
    np.random.seed(42)
    accel_data = [np.random.randn(n_samples, n_features) for _ in range(100)]
    labels = np.random.choice([0, 1], size=100)  # Binary labels
    return accel_data, labels

# Function to perform wavelet packet decomposition and extract features
def extract_wavelet_packet_features(accel_data, wavelet='db4', max_level=2):
    """
    Extract features using wavelet packet decomposition.
    """
    accel_array = np.array(accel_data)
    x, y, z = accel_array[:, 0], accel_array[:, 1], accel_array[:, 2]

    def decompose_and_extract(axis_data):
        wp = pywt.WaveletPacket(data=axis_data, wavelet=wavelet, maxlevel=max_level)
        features = {}
        
        # Iterate over all nodes in the wavelet packet tree
        for node in wp.get_level(max_level, order='natural'):
            node_data = node.data
            features[f'{node.path}_mean'] = np.mean(node_data)
            features[f'{node.path}_std'] = np.std(node_data)
            features[f'{node.path}_energy'] = np.sum(np.square(node_data))
        return features

    # Extract features for each axis
    features_x = decompose_and_extract(x)
    features_y = decompose_and_extract(y)
    features_z = decompose_and_extract(z)
    
    # Combine features from all axes
    features = {**features_x, **features_y, **features_z}
    return features

# Evaluate the impact of unequal splits on accuracy
def evaluate_unequal_splits(accel_data, labels, wavelet='db4', max_level_range=[2, 3, 4]):
    results = []
    
    for max_level in max_level_range:
        features = [extract_wavelet_packet_features(data, wavelet, max_level) for data in accel_data]
        feature_df = pd.DataFrame(features)
        
        # Train/test split
        X_train, X_test, y_train, y_test = train_test_split(feature_df, labels, test_size=0.3, random_state=42)
        
        # Train a simple Random Forest Classifier
        model = RandomForestClassifier(random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluate accuracy
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results.append({'max_level': max_level, 'accuracy': accuracy})
    
    return pd.DataFrame(results)



# Calling wavelet

In [8]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

with open('/kaggle/working/LowPass_train_45.json', 'r') as f:
    data = json.load(f)


df = pd.DataFrame(data)

In [7]:
df.head()

,subject,chunk_id,ts,accel,label
0,80,0,"[1725411020.0856903, 1725411020.0897202, 17254...","[[-82.5083999633789, -56.03909683227539, 1000....",0
1,14,1,"[1725496482.262797, 1725496482.266831, 1725496...","[[-20.9507999420166, 65.57850646972656, 1027.9...",1
2,300,2,"[1726611768.779811, 1726611768.783809, 1726611...","[[108.17430114746094, 173.58509826660156, 968....",1
3,193,3,"[1725492595.229943, 1725492595.233981, 1725492...","[[15.732599258422852, 70.74210357666016, 1001....",1
4,9,4,"[1725486791.803703, 1725486791.807745, 1725486...","[[-29.815500259399414, 47.252403259277344, 100...",0


In [24]:
features_df2.shape,len(df2['label']),final_df2.shape

((1183, 99), 1183, (1183, 100))

In [23]:
final_df2 = pd.concat([features_df2.reset_index(drop=True), df2['label'].reset_index(drop=True)], axis=1)


In [26]:
feature_list = df['accel'].apply(extract_wavelet_features).tolist()
# feature_list1 = df1['accel'].apply(extract_wavelet_features).tolist()
# feature_list2 = df2['accel'].apply(extract_wavelet_features).tolist()

# features_df1 = pd.DataFrame(feature_list1)
# features_df2 = pd.DataFrame(feature_list2)

# Combine features with labels
final_df = pd.concat([features_df, df['label']], axis=1)
# final_df1 = pd.concat([features_df1, df1['label']], axis=1)
# final_df2 = pd.concat([features_df2, df2['label']], axis=1)

# final_df = pd.concat([features_df, df['labels']], axis=1)

print(final_df.head(),final_df.shape)
# print(final_df1.head(),final_df1.shape)
# print(final_df2.head(),final_df2.shape)

   cA_mean_x_level0  cA_std_x_level0  cA_max_x_level0  cA_min_x_level0  \
0       -164.618457         0.604509      -162.780599      -166.840873   
1       -313.811041       122.033092        58.848554      -664.304205   
2        218.086563         1.702307       223.668579       214.183751   
3         32.078675         0.729502        34.458902        29.929647   
4        -60.829468         1.212052       -55.629101       -65.011295   

   cA_energy_x_level0  cA_skewness_x_level0  cA_kurtosis_x_level0  \
0        1.707275e+07             -0.076523             -0.001264   
1        7.142275e+07             -1.054212              2.516957   
2        2.996573e+07              0.572028              0.277138   
3        6.486314e+05              0.004732              0.795969   
4        2.332067e+06              0.026571              0.925110   

   cA_median_x_level0  cA_iqr_x_level0  cA_entropy_x_level0  ...  \
0         -164.616285         0.847286             6.445713  ...   
1   

In [18]:
print(final_df.head(),final_df.shape)

   cA_mean_x_level0  cA_std_x_level0  cA_max_x_level0  cA_min_x_level0  \
0       -164.618457         0.604509      -162.780599      -166.840873   
1       -313.811041       122.033092        58.848554      -664.304205   
2        218.086563         1.702307       223.668579       214.183751   
3         32.078675         0.729502        34.458902        29.929647   
4        -60.829468         1.212052       -55.629101       -65.011295   

   cA_energy_x_level0  cA_skewness_x_level0  cA_kurtosis_x_level0  \
0        1.707275e+07             -0.076523             -0.001264   
1        7.142275e+07             -1.054212              2.516957   
2        2.996573e+07              0.572028              0.277138   
3        6.486314e+05              0.004732              0.795969   
4        2.332067e+06              0.026571              0.925110   

   cA_median_x_level0  cA_iqr_x_level0  cA_entropy_x_level0  ...  \
0         -164.616285         0.847286             6.445713  ...   
1   

## Train RFC

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score

X_train = final_df1.drop(columns='label')
y_train = final_df1['label']
X_test = final_df2.drop(columns='label')
y_test = final_df2['label']

# Split the dataset into features and labels
# X = final_df.drop(columns='labels')
# y = final_df['labels']

# Split into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4,stratify=y)

# Step 3: Scale Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
# model.fit(X_train, y_train)
model.fit(X_train,y_train)
# model.fit(X, y)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calculate and print the balanced accuracy
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc}")

[[431   4]
 [  0 748]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       435
           1       0.99      1.00      1.00       748

    accuracy                           1.00      1183
   macro avg       1.00      1.00      1.00      1183
weighted avg       1.00      1.00      1.00      1183

Balanced Accuracy: 0.9954022988505746


# Train ANN

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Generate Synthetic Data
# 2 features and binary labels (0 or 1)
# np.random.seed(42)
# X = final_df.drop(columns='label')
# y = final_df['label']

# Step 2: Split Data into Train/Test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = final_df1.drop(columns='label')
y_train = final_df1['label']
X_test = final_df2.drop(columns='label')
y_test = final_df2['label']

# Step 3: Scale Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 4: Build the ANN Model
ann_model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),  # Input layer with 32 neurons
    Dropout(0.2),  # Dropout for regularization
    Dense(16, activation='relu'),  # Hidden layer with 16 neurons
    Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
])

# Step 5: Compile the Model
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Train the Model
history = ann_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Step 7: Evaluate the Model
y_pred = (ann_model.predict(X_test) > 0.5).astype(int)  # Predict and convert probabilities to binary classes
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6305 - loss: 0.6148 - val_accuracy: 0.8266 - val_loss: 0.4162
Epoch 2/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8479 - loss: 0.3723 - val_accuracy: 0.8605 - val_loss: 0.3318
Epoch 3/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8852 - loss: 0.2827 - val_accuracy: 0.8732 - val_loss: 0.2957
Epoch 4/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9062 - loss: 0.2351 - val_accuracy: 0.8837 - val_loss: 0.2663
Epoch 5/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9090 - loss: 0.2203 - val_accuracy: 0.8996 - val_loss: 0.2518
Epoch 6/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9240 - loss: 0.1855 - val_accuracy: 0.9070 - val_loss: 0.2397
Epoch 7/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9371 - loss: 0.1606 - val_accuracy: 0.9218 - val_loss: 0.2312
Epoch 8/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9388 - loss: 0.1446 - val_accuracy: 0.9239 - val_

# Testing

In [13]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

with open('/kaggle/input/icassp-person-in-bed-track-1/test.json', 'r') as f:
    data_test = json.load(f)


df_test = pd.DataFrame(data_test)

In [14]:
df_test.head()

,subject,chunk_id,accel,ts
0,91,0,"[[-16.750499725341797, 221.14559936523438, 978...","[1725487568.2857084, 1725487568.2897425, 17254..."
1,91,1,"[[12.323999404907227, -6.38040018081665, 1004....","[1725488230.4562604, 1725488230.460306, 172548..."
2,91,2,"[[39.14820098876953, 99.29010009765625, 989.72...","[1725487762.2033925, 1725487762.2074304, 17254..."
3,91,3,"[[-19.905601501464844, 241.90139770507812, 973...","[1725487415.7801855, 1725487415.7842104, 17254..."
4,91,4,"[[-11.54010009765625, 169.252197265625, 989.16...","[1725488018.8184955, 1725488018.8225424, 17254..."


In [15]:
import json
import numpy as np
from scipy.signal import butter, filtfilt

# Define a function to create a low-pass Butterworth filter
def butter_lowpass(cutoff, fs, order=4):
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist  # Normalize cutoff frequency
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# Apply the low-pass filter to data
def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order)
    return filtfilt(b, a, data)

# Function to process a single subject
def process_subject(subject_data, cutoff_freq=45.0, sampling_rate=250):
    # Extract accelerometer data and timestamps
    accel_data = np.array(subject_data['accel'])
    ts_data = np.array(subject_data['ts'])
    
    # Apply low-pass filter to each axis (x, y, z)
    filtered_accel = np.array([butter_lowpass_filter(accel_data[:, i], cutoff_freq, sampling_rate) for i in range(3)]).T
    
    # Return processed subject data without downsampling
    return {
        'subject': subject_data['subject'],
        'accel': filtered_accel.tolist(),
        'ts': ts_data.tolist(),
        # 'labels': subject_data['label']  # Keep original labels
    }

# Load the JSON file
with open('/kaggle/input/icassp-person-in-bed-track-1/test.json', 'r') as f:
    data = json.load(f)

# Process each subject independently
processed_data = [process_subject(subject_data) for subject_data in data]

# Save the processed data back to JSON
with open('LowPass_test_45.json', 'w') as f:
    json.dump(processed_data, f)

print("Low-pass filtering (50 Hz) complete without downsampling.")

Low-pass filtering (50 Hz) complete without downsampling.


In [16]:
print(df_test.head())
print(df_test.shape)
l = df_test['subject']
print(l.unique())

   subject  chunk_id                                              accel  \
0       91         0  [[-16.750499725341797, 221.14559936523438, 978...   
1       91         1  [[12.323999404907227, -6.38040018081665, 1004....   
2       91         2  [[39.14820098876953, 99.29010009765625, 989.72...   
3       91         3  [[-19.905601501464844, 241.90139770507812, 973...   
4       91         4  [[-11.54010009765625, 169.252197265625, 989.16...   

                                                  ts  
0  [1725487568.2857084, 1725487568.2897425, 17254...  
1  [1725488230.4562604, 1725488230.460306, 172548...  
2  [1725487762.2033925, 1725487762.2074304, 17254...  
3  [1725487415.7801855, 1725487415.7842104, 17254...  
4  [1725488018.8184955, 1725488018.8225424, 17254...  
(1538, 4)
[ 91 108 130 127 157 194 151 171 367]


In [15]:
len(df_test['accel'].iloc[1])

2500

In [17]:
with open('/kaggle/working/LowPass_test_45.json', 'r') as f:
    processed_data_test = json.load(f)
processed_df_test = pd.DataFrame(processed_data_test)

feature_list = processed_df_test['accel'].apply(extract_wavelet_features).tolist()
# feature_list = df_test['accel'].apply(extract_wavelet_packet_features).tolist()
features_df = pd.DataFrame(feature_list)
final_df_test = features_df

print(final_df_test.head())

# feature_list = df_test['accel'].apply(extract_wavelet_features).tolist()
# # feature_list = df_test['accel'].apply(extract_wavelet_packet_features).tolist()
# features_df = pd.DataFrame(feature_list)
# final_df_test = features_df

# print(final_df_test.head())

   cA_mean_x_level0  cA_std_x_level0  cA_max_x_level0  cA_min_x_level0  \
0        -32.305045         0.802624       -29.822623       -35.445396   
1         24.847054         0.406827        26.120423        23.580158   
2        -27.811161        35.624847       170.000804      -253.563583   
3        -39.736626         0.619029       -38.304170       -42.560620   
4        -21.670079         0.822990       -18.870422       -23.994127   

   cA_energy_x_level0  cA_skewness_x_level0  cA_kurtosis_x_level0  \
0        6.578839e+05              0.065643              0.631282   
1        3.890512e+05              0.107702              0.042736   
2        1.286832e+06              0.832988             10.165146   
3        9.950111e+05             -0.922166              2.123271   
4        2.962699e+05              0.168244              0.311575   

   cA_median_x_level0  cA_iqr_x_level0  cA_entropy_x_level0  ...  \
0          -32.358610         1.033436             6.445411  ...   
1   

In [18]:
final_df.shape,final_df_test.shape

((5911, 100), (1538, 99))

In [19]:
# Split the dataset into features and labels
# X = final_df.drop(columns='label')
# y = final_df['label']

# Make predictions on the test set
y_pred_test = model.predict(final_df_test)
print(len(y_pred_test))
# Extract features from the test data
test_features = []
chunk_ids = []

for entry in data_test:
    chunk_id = entry['chunk_id']
    accel_data = entry['accel']  # List of 3-axis accelerometer readings

    # Extract periodic features using the defined function
    # features = extract_periodic_features(accel_data)
    # features['chunk_id'] = chunk_id
    # test_features.append(features)
    chunk_ids.append(chunk_id)


# Create the submission DataFrame
submission_df = pd.DataFrame({
    'chunk_id': chunk_ids,
    'label': y_pred_test
})

# Save the submission to a CSV file
submission_file = 'Track1_lvl2_butterworth_45Hz.csv'
submission_df.to_csv(submission_file, index=False)

print(f"Submission file saved as {submission_file}")

1538
Submission file saved as Track1_lvl2_butterworth_45Hz.csv


In [19]:
len(submission_df),len(y_pred_test)

(1538, 1538)